### Importing all the important libraries

In [1]:
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from IPython.display import display
import torch
import unicodedata
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, SeamlessM4TForTextToText, AutoProcessor
from sentence_transformers import SentenceTransformer, util

### Set GPU

#### Mac

In [2]:
# for mac
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        details = tf.config.experimental.get_device_details(gpu)
        print("GPU details: ", details)
else:
    print("No GPU found. Using CPU.")

# set GPU device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}
Using device: mps


#### Window

In [3]:
# for window
print("Tensorflow GPUs: ", tf.config.list_physical_devices('GPU'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using PyTorch device:", device)
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

Tensorflow GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using PyTorch device: cpu


### Functions

In [3]:
# function to save gen df
def save_gen_df(df, df_name):
    df.to_csv(f"gen/{df_name}.csv", index=False)

In [4]:
# function to save tmp df
def save_tmp_df(df, df_name):
    df.to_csv(f"tmp/{df_name}.csv", index=False)

In [5]:
# function to load generated df
def load_gen_df(df_name):
    return pd.read_csv(f"gen/{df_name}.csv", header=0)

In [6]:
# function to load tmp df
def load_tmp_df(df_name):
    return pd.read_csv(f"tmp/{df_name}.csv", header=0)

### Set settings

In [7]:
tqdm.pandas()

### 1. Dataset Loading

This step involves loading the datasets `myXNLI` and `ALT Corpus` into pandas DataFrames. 
The English and Burmese datasets from the ALT Corpus are combined to create a bilingual parallel corpus.

In [6]:
# Load myXNLI dataset
myxnli_path = './data/myXNLI.train.tsv'  # Path to the file
myxnli_data = pd.read_csv(myxnli_path, sep='\t', header=0)
print(f"myXNLI dataset loaded successfully with {len(myxnli_data)} records.")
display(myxnli_data.head())  # Display the first few rows of the dataset

myXNLI dataset loaded successfully with 392702 records.


,genre,label,sentence1_en,sentence2_en,sentence1_my,sentence2_my
0,government,neutral,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,သဘောတရားအရ ခရင်မ်စိမ်ခြင်းတွင် အခြေခံအတိုင်းအတ...,ထုတ်ကုန်နှင့် ပထဝီဝင်အနေအထားသည် ခရင်မ် skimmin...
1,telephone,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,ရာသီအတွင်း မင်းသိတယ်၊ မင်းရဲ့အဆင့်ကို ငါ ခန့်မ...,လူတွေပြန်ခေါ်ရင် အောက်ပါအဆင့်အထိ ဆုံးရှုံးသွား...
2,fiction,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,ကျွန်ုပ်တို့၏နံပါတ်တစ်ခုသည် သင့်ညွှန်ကြားချက်မ...,ကျွန်ုပ်၏အဖွဲ့သားတစ်ဦးသည် သင်၏အမိန့်စာများကို ...
3,fiction,entailment,How do you know? All this is their information...,This information belongs to them.,သင်ဘယ်လိုသိသလဲ? ဒါတွေအားလုံးဟာ သူတို့ရဲ့ အချက်...,ဒီအချက်အလက်က သူတို့ပိုင်တယ်။
4,telephone,neutral,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,ဟုတ်တယ် ငါမင်းကိုပြောပြမယ် ဒီတင်းနစ်ဖိနပ်တချို...,တင်းနစ်ဖိနပ်များသည် ဈေးနှုန်းအမျိုးမျိုးရှိသည်။


In [7]:
# Load ALT English data
alt_en_path = './data/ALT_data_en.txt'  # Path to the English ALT corpus
alt_en_data = pd.read_csv(alt_en_path, sep='\t', header=None, names=["ID", "English_Sentence"])
print(f"ALT English dataset loaded successfully with {len(alt_en_data)} records.")
display(alt_en_data.head())

ALT English dataset loaded successfully with 19908 records.


,ID,English_Sentence
0,SNT.80188.1,Italy have defeated Portugal 31-5 in Pool C of...
1,SNT.80188.2,Andrea Masi opened the scoring in the fourth m...
2,SNT.80188.3,Despite controlling the game for much of the f...
3,SNT.80188.4,Portugal never gave up and David Penalva score...
4,SNT.80188.5,Italy led 16-5 at half time but were matched b...


In [8]:
# Load ALT Burmese data
alt_my_path = './data/ALT_data_my.txt'  # Path to the Burmese ALT corpus
alt_my_data = pd.read_csv(alt_my_path, sep='\t', header=None, names=["ID", "Burmese_Sentence"])
print(f"ALT Burmese dataset loaded successfully with {len(alt_my_data)} records.")
display(alt_my_data.head())

ALT Burmese dataset loaded successfully with 19265 records.


,ID,Burmese_Sentence
0,SNT.80188.1,ပြင်သစ်နိုင်ငံ ပါရီမြို့ ပါ့ဒက်စ် ပရင့်စက် ၌ ၂...
1,SNT.80188.2,အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
2,SNT.80188.3,ပထမ တစ်ဝက် ၏ တော်တော်များများ အတွက် ကစားပွဲ ကိ...
3,SNT.80188.4,ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
4,SNT.80188.5,အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...


In [9]:
# Combine ALT datasets (if IDs match)
alt_combined = pd.merge(alt_en_data, alt_my_data, on="ID")
print(f"ALT combined dataset created successfully with {len(alt_combined)} records.")
display(alt_combined.head())

ALT combined dataset created successfully with 19173 records.


,ID,English_Sentence,Burmese_Sentence
0,SNT.80188.1,Italy have defeated Portugal 31-5 in Pool C of...,ပြင်သစ်နိုင်ငံ ပါရီမြို့ ပါ့ဒက်စ် ပရင့်စက် ၌ ၂...
1,SNT.80188.2,Andrea Masi opened the scoring in the fourth m...,အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
2,SNT.80188.3,Despite controlling the game for much of the f...,ပထမ တစ်ဝက် ၏ တော်တော်များများ အတွက် ကစားပွဲ ကိ...
3,SNT.80188.4,Portugal never gave up and David Penalva score...,ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
4,SNT.80188.5,Italy led 16-5 at half time but were matched b...,အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...


### 2. Data Cleaning
This step focuses on cleaning the datasets to prepare them for further processing. The cleaning operations include:
1. Removing duplicate entries.
2. Handling missing values.
3. Removing non-standard characters or symbols unrelated to the Burmese or English language.
4. Ensuring consistent formatting.

The cleaned datasets will be ready for normalization and tokenization in the next steps.

In [10]:
# Cleaning myXNLI dataset
print("Cleaning myXNLI dataset...")
myxnli_cleaned = myxnli_data.drop_duplicates()  # Remove duplicates
myxnli_cleaned = myxnli_cleaned.dropna()  # Remove rows with missing values
#myxnli_cleaned = myxnli_cleaned.replace(r'[^\w\s]', '', regex=True)  # Remove non-standard characters
print(f"myXNLI dataset cleaned successfully.")
print(f"Original Records: {len(myxnli_data)}.")
print(f"Remaining records: {len(myxnli_cleaned)}.")
display(myxnli_cleaned.head())

Cleaning myXNLI dataset...
myXNLI dataset cleaned successfully.
Original Records: 392702.
Remaining records: 392682.


,genre,label,sentence1_en,sentence2_en,sentence1_my,sentence2_my
0,government,neutral,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...,သဘောတရားအရ ခရင်မ်စိမ်ခြင်းတွင် အခြေခံအတိုင်းအတ...,ထုတ်ကုန်နှင့် ပထဝီဝင်အနေအထားသည် ခရင်မ် skimmin...
1,telephone,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...,ရာသီအတွင်း မင်းသိတယ်၊ မင်းရဲ့အဆင့်ကို ငါ ခန့်မ...,လူတွေပြန်ခေါ်ရင် အောက်ပါအဆင့်အထိ ဆုံးရှုံးသွား...
2,fiction,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...,ကျွန်ုပ်တို့၏နံပါတ်တစ်ခုသည် သင့်ညွှန်ကြားချက်မ...,ကျွန်ုပ်၏အဖွဲ့သားတစ်ဦးသည် သင်၏အမိန့်စာများကို ...
3,fiction,entailment,How do you know? All this is their information...,This information belongs to them.,သင်ဘယ်လိုသိသလဲ? ဒါတွေအားလုံးဟာ သူတို့ရဲ့ အချက်...,ဒီအချက်အလက်က သူတို့ပိုင်တယ်။
4,telephone,neutral,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.,ဟုတ်တယ် ငါမင်းကိုပြောပြမယ် ဒီတင်းနစ်ဖိနပ်တချို...,တင်းနစ်ဖိနပ်များသည် ဈေးနှုန်းအမျိုးမျိုးရှိသည်။


In [11]:
# save cleaned myXNLI dataset
save_gen_df(myxnli_cleaned, "myxnli_cleaned")

In [12]:
# Cleaning ALT English data
print("Cleaning ALT English dataset...")
alt_en_cleaned = alt_en_data.drop_duplicates()  # Remove duplicates
alt_en_cleaned = alt_en_cleaned.dropna()  # Remove rows with missing values
alt_en_cleaned["English_Sentence"] = alt_en_cleaned["English_Sentence"].replace(r'[^\w\s]', '', regex=True)  # Remove non-standard characters
print(f"ALT English dataset cleaned successfully.")
print(f"Original records: {len(alt_en_data)}.")
print(f"Remaining records: {len(alt_en_cleaned)}.")
display(alt_en_cleaned.head())

Cleaning ALT English dataset...
ALT English dataset cleaned successfully.
Original records: 19908.
Remaining records: 19908.


,ID,English_Sentence
0,SNT.80188.1,Italy have defeated Portugal 315 in Pool C of ...
1,SNT.80188.2,Andrea Masi opened the scoring in the fourth m...
2,SNT.80188.3,Despite controlling the game for much of the f...
3,SNT.80188.4,Portugal never gave up and David Penalva score...
4,SNT.80188.5,Italy led 165 at half time but were matched by...


In [13]:
# Cleaning ALT Burmese data
print("Cleaning ALT Burmese dataset...")
alt_my_cleaned = alt_my_data.drop_duplicates()  # Remove duplicates
alt_my_cleaned = alt_my_cleaned.dropna()  # Remove rows with missing values
#alt_my_cleaned["Burmese_Sentence"] = alt_my_cleaned["Burmese_Sentence"].replace(r'[^\w\s]', '', regex=True)  # Remove non-standard characters
print(f"ALT Burmese dataset cleaned successfully.")
print(f"Original records: {len(alt_my_data)}")
print(f"Remaining records: {len(alt_my_cleaned)}")
display(alt_my_cleaned.head())

Cleaning ALT Burmese dataset...
ALT Burmese dataset cleaned successfully.
Original records: 19265
Remaining records: 19258


,ID,Burmese_Sentence
0,SNT.80188.1,ပြင်သစ်နိုင်ငံ ပါရီမြို့ ပါ့ဒက်စ် ပရင့်စက် ၌ ၂...
1,SNT.80188.2,အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
2,SNT.80188.3,ပထမ တစ်ဝက် ၏ တော်တော်များများ အတွက် ကစားပွဲ ကိ...
3,SNT.80188.4,ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
4,SNT.80188.5,အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...


In [14]:
# Combine cleaned ALT datasets
print("Cleaning combined ALT dataset...")
alt_combined_cleaned = pd.merge(alt_en_cleaned, alt_my_cleaned, on="ID")
print(f"Combined ALT dataset cleaned successfully.")
print(f"Original records: {len(alt_combined)}")
print(f"Remaining records: {len(alt_combined_cleaned)}")
display(alt_combined_cleaned.head())

Cleaning combined ALT dataset...
Combined ALT dataset cleaned successfully.
Original records: 19173
Remaining records: 19166


,ID,English_Sentence,Burmese_Sentence
0,SNT.80188.1,Italy have defeated Portugal 315 in Pool C of ...,ပြင်သစ်နိုင်ငံ ပါရီမြို့ ပါ့ဒက်စ် ပရင့်စက် ၌ ၂...
1,SNT.80188.2,Andrea Masi opened the scoring in the fourth m...,အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
2,SNT.80188.3,Despite controlling the game for much of the f...,ပထမ တစ်ဝက် ၏ တော်တော်များများ အတွက် ကစားပွဲ ကိ...
3,SNT.80188.4,Portugal never gave up and David Penalva score...,ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
4,SNT.80188.5,Italy led 165 at half time but were matched by...,အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...


In [15]:
# save cleaned combined ALT dataset
save_gen_df(alt_combined_cleaned, "alt_combined_cleaned")

### 3. Data Normalization
This step normalizes the text data to ensure consistency across datasets. The normalization process includes:
1. Applying Unicode normalization to handle encoding inconsistencies.
2. Standardizing text formatting by converting all text to lowercase and standardizing punctuation.
3. Normalizing diacritical marks and stacked consonants in the Burmese text to improve text representation.

In [16]:
# Function to normalize text
def normalize_text(text):
    if pd.isnull(text):
        return text  # Skip null values
    # Apply Unicode normalization
    normalized_text = unicodedata.normalize('NFKC', text)
    # Convert to lowercase
    normalized_text = normalized_text.lower()
    # Standardize punctuation (e.g., replace unusual punctuation marks)
    normalized_text = normalized_text.replace('“', '"').replace('”', '"').replace('’', "'")
    return normalized_text

In [17]:
# Function to normalize Burmese text (handles diacritical marks and stacked consonants)
def normalize_burmese(text):
    if pd.isnull(text):
        return text  # Skip null values
    normalized_text = unicodedata.normalize('NFKC', text)
    # Additional Burmese-specific normalization can be added here if needed
    return normalized_text

In [18]:
# Normalize myXNLI cleaned dataset
print("Normalizing myXNLI dataset...")
myxnli_normalized = load_gen_df("myxnli_cleaned")

# Normalize English columns
myxnli_normalized["sentence1_en"] = myxnli_normalized["sentence1_en"].apply(normalize_text)
myxnli_normalized["sentence2_en"] = myxnli_normalized["sentence2_en"].apply(normalize_text)

# Normalize Burmese columns
myxnli_normalized["sentence1_my"] = myxnli_normalized["sentence1_my"].apply(normalize_burmese)
myxnli_normalized["sentence2_my"] = myxnli_normalized["sentence2_my"].apply(normalize_burmese)

print(f"myXNLI dataset normalized successfully.")
display(myxnli_normalized.head())

Normalizing myXNLI dataset...
myXNLI dataset normalized successfully.


,genre,label,sentence1_en,sentence2_en,sentence1_my,sentence2_my
0,government,neutral,conceptually cream skimming has two basic dime...,product and geography are what make cream skim...,သဘောတရားအရ ခရင်မ်စိမ်ခြင်းတွင် အခြေခံအတိုင်းအတ...,ထုတ်ကုန်နှင့် ပထဝီဝင်အနေအထားသည် ခရင်မ် skimmin...
1,telephone,entailment,you know during the season and i guess at at y...,you lose the things to the following level if ...,ရာသီအတွင်း မင်းသိတယ်၊ မင်းရဲ့အဆင့်ကို ငါ ခန့်မ...,လူတွေပြန်ခေါ်ရင် အောက်ပါအဆင့်အထိ ဆုံးရှုံးသွား...
2,fiction,entailment,one of our number will carry out your instruct...,a member of my team will execute your orders w...,ကျွန်ုပ်တို့၏နံပါတ်တစ်ခုသည် သင့်ညွှန်ကြားချက်မ...,ကျွန်ုပ်၏အဖွဲ့သားတစ်ဦးသည် သင်၏အမိန့်စာများကို ...
3,fiction,entailment,how do you know? all this is their information...,this information belongs to them.,သင်ဘယ်လိုသိသလဲ? ဒါတွေအားလုံးဟာ သူတို့ရဲ့ အချက်...,ဒီအချက်အလက်က သူတို့ပိုင်တယ်။
4,telephone,neutral,yeah i tell you what though if you go price so...,the tennis shoes have a range of prices.,ဟုတ်တယ် ငါမင်းကိုပြောပြမယ် ဒီတင်းနစ်ဖိနပ်တချို...,တင်းနစ်ဖိနပ်များသည် ဈေးနှုန်းအမျိုးမျိုးရှိသည်။


In [19]:
# save normalized myXNLI dataset
save_gen_df(myxnli_normalized, "myxnli_normalized")

In [20]:
# Normalize combined ALT cleaned dataset
print("Normalizing combined ALT dataset...")
alt_combined_normalized = load_gen_df("alt_combined_cleaned")
alt_combined_normalized["English_Sentence"] = alt_combined_normalized["English_Sentence"].apply(normalize_text)
alt_combined_normalized["Burmese_Sentence"] = alt_combined_normalized["Burmese_Sentence"].apply(normalize_burmese)
print(f"Combined ALT dataset normalized successfully.")
display(alt_combined_normalized.head())

Normalizing combined ALT dataset...
Combined ALT dataset normalized successfully.


,ID,English_Sentence,Burmese_Sentence
0,SNT.80188.1,italy have defeated portugal 315 in pool c of ...,ပြင်သစ်နိုင်ငံ ပါရီမြို့ ပါ့ဒက်စ် ပရင့်စက် ၌ ၂...
1,SNT.80188.2,andrea masi opened the scoring in the fourth m...,အန်ဒရီယာ မာစီ သည် အီတလီ အတွက် စမ်းသပ်မှု တစ်ခု...
2,SNT.80188.3,despite controlling the game for much of the f...,ပထမ တစ်ဝက် ၏ တော်တော်များများ အတွက် ကစားပွဲ ကိ...
3,SNT.80188.4,portugal never gave up and david penalva score...,ပေါ်တူဂီ သည် ဘယ်သောအခါမှ စွန့်လွှတ်မှု မရှိခဲ့...
4,SNT.80188.5,italy led 165 at half time but were matched by...,အီတလီ သည် ပထမပိုင်း ၌ ၁၆-၅ ဖြင့် ဦးဆောင်ခဲ့ သေ...


In [21]:
# save normalized ALT dataset
save_gen_df(alt_combined_normalized, "alt_combined_normalized")

### 4. Back-Translation Augmentation
This step applies back-translation augmentation to the `myXNLI` and `ALT Combined` datasets. Models (`facebook/nllb-200-distilled-600M` and `facebook/hf-seamless-m4t-large`) are used to generate synthetic data. 
Results are stored in additional columns for evaluation and comparison.

In [9]:
# load normalized myXNLI and Combined ALT  dataset
myxnli_back_translated = load_gen_df("myxnli_normalized")
alt_combined_back_translated = load_gen_df("alt_combined_normalized")

#### facebook/nllb-200-distilled-600M

In [7]:
# Load NLLB model and tokenizer
nllb_model_name = "facebook/nllb-200-distilled-600M"
nllb_model = AutoModelForSeq2SeqLM.from_pretrained(nllb_model_name).to(device)
nllb_tokenizer = AutoTokenizer.from_pretrained(nllb_model_name)

In [8]:
# Function for back-translation using NLLB
def back_translate_nllb(text, src_lang="eng_Latn", tgt_lang="mya_Mymr"):
    try:
        # Forward translation: English -> Burmese
        inputs = nllb_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
        translated_tokens = nllb_model.generate(
            **inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(tgt_lang)
        )
        translated_text = nllb_tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        
        # Backward translation: Burmese -> English
        back_inputs = nllb_tokenizer(translated_text, return_tensors="pt", truncation=True, padding=True).to(device)
        back_translated_tokens = nllb_model.generate(
            **back_inputs, forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids(src_lang)
        )
        back_translated_text = nllb_tokenizer.batch_decode(back_translated_tokens, skip_special_tokens=True)[0]
        
        return translated_text, back_translated_text
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return None, None

##### myXNLI dataset

In [11]:
# Apply back-translation to the sentences 1 in myXNLI
print("Applying back-translation to myXNLI dataset (sentences 1)...")
myxnli_nllb_back_translated_start_index_1 = 0

with open("tmp/myxnli_nllb_back_translated_1.csv", mode="a", encoding="utf-8") as f:
    if myxnli_nllb_back_translated_start_index_1 == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(myxnli_back_translated.iloc[myxnli_nllb_back_translated_start_index_1:].iterrows(), 
                           total=len(myxnli_back_translated) - myxnli_nllb_back_translated_start_index_1):
        original = row["sentence1_en"]
        translated, back_translated = back_translate_nllb(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'")
        translated = translated.replace('"', "'")
        back_translated = back_translated.replace('"', "'")
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")


Applying back-translation to myXNLI dataset (sentences 1)...


  0%|          | 0/203657 [00:00<?, ?it/s]

In [ ]:
# set and display back-translated myXNLI dataset (sentences 1)
tmp_myxnli_nllb_back_translated_1 = load_tmp_df("myxnli_nllb_back_translated_1")
myxnli_back_translated["nllb_translated_s1"] = tmp_myxnli_nllb_back_translated_1["translated"]
myxnli_back_translated["nllb_back_translated_s1"] = tmp_myxnli_nllb_back_translated_1["back_translated"]
display(myxnli_back_translated.head())

In [ ]:
# Apply back-translation to the sentences 2 in myXNLI
print("Applying back-translation to myXNLI dataset (sentences 2)...")
myxnli_nllb_back_translated_start_index_2 = 97494
with open("tmp/myxnli_nllb_back_translated_2.csv", mode="w", encoding="utf-8") as f:
    if myxnli_nllb_back_translated_start_index_2 == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(myxnli_back_translated.iloc[myxnli_nllb_back_translated_start_index_2:].iterrows(), 
                       total=len(myxnli_back_translated) - myxnli_nllb_back_translated_start_index_2):
        original = row["sentence2_en"]
        translated, back_translated = back_translate_nllb(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'") if original is not None else None
        translated = translated.replace('"', "'") if translated is not None else None
        back_translated = back_translated.replace('"', "'") if back_translated is not None else None
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")

Applying back-translation to myXNLI dataset (sentences 2)...


  0%|          | 0/295188 [00:00<?, ?it/s]

In [ ]:
# set and display back-translated myXNLI dataset (sentences 2)
tmp_myxnli_nllb_back_translated_2 = load_tmp_df("myxnli_nllb_back_translated_2")
myxnli_back_translated["nllb_translated_s2"] = tmp_myxnli_nllb_back_translated_2["translated"]
myxnli_back_translated["nllb_back_translated_s2"] = tmp_myxnli_nllb_back_translated_2["back_translated"]
display(myxnli_back_translated.head())

##### ALT dataset

In [ ]:
# Apply back-translation to the alt combined dataset
print("Applying back-translation to alt combined dataset...")
alt_combined_nllb_back_translated_start_index = 0
with open("tmp/alt_combined_nllb_back_translated.csv", mode="a", encoding="utf-8") as f:
    if alt_combined_nllb_back_translated_start_index == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(alt_combined_back_translated.iloc[alt_combined_nllb_back_translated_start_index:].iterrows(),
                       total=len(alt_combined_back_translated) - alt_combined_nllb_back_translated_start_index):
        original = row["English_Sentence"]
        translated, back_translated = back_translate_nllb(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'")
        translated = translated.replace('"', "'")
        back_translated = back_translated.replace('"', "'")
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")

Applying back-translation to alt combined dataset...


  0%|          | 0/19166 [00:00<?, ?it/s]

In [ ]:
# set and display back-translated combined ALT dataset
tmp_alt_combined_nllb_back_translated = load_tmp_df("alt_combined_nllb_back_translated")
alt_combined_back_translated["nllb_translated"] = tmp_alt_combined_nllb_back_translated["translated"]
alt_combined_back_translated["nllb_back_translated"] = tmp_alt_combined_nllb_back_translated["back_translated"]
display(alt_combined_back_translated.head())

#### facebook/hf-seamless-m4t-large

In [6]:
# Load seamless m4t model and processor
seamless_m4t_model_name = "facebook/hf-seamless-m4t-large"
seamless_m4t_model = SeamlessM4TForTextToText.from_pretrained(seamless_m4t_model_name).to(device)
seamless_m4t_processor = AutoProcessor.from_pretrained(seamless_m4t_model_name)

In [13]:
# Function for back-translation using seamless_m4t
def back_translate_seamless_m4t(text, src_lang="eng", tgt_lang="mya"):
    try:
        # Forward translation: English -> Burmese
        text_inputs = seamless_m4t_processor(text, src_lang=src_lang, return_tensors="pt", padding=True).to(device)
        output_tokens = seamless_m4t_model.generate(**text_inputs, tgt_lang=tgt_lang)
        translated_text = seamless_m4t_processor.decode(output_tokens[0].tolist(), skip_special_tokens=True)
        
        # Backward translation: Burmese -> English
        back_text_inputs = seamless_m4t_processor(translated_text, src_lang=tgt_lang, return_tensors="pt", padding=True).to(device)
        back_output_tokens = seamless_m4t_model.generate(**back_text_inputs, tgt_lang=src_lang)
        back_translated_text = seamless_m4t_processor.decode(back_output_tokens[0].tolist(), skip_special_tokens=True)
        
        return translated_text, back_translated_text
    except Exception as e:
        print(f"Error processing text: {text}. Error: {e}")
        return None, None

##### myXNLI dataset

In [ ]:
# Apply back-translation to the sentences 1 in myXNLI
print("Applying back-translation to myXNLI dataset (sentences 1)...")
myxnli_seamless_m4t_back_translated_start_index_1 = 0
with open("tmp/myxnli_seamless_m4t_back_translated_1.csv", mode="a", encoding="utf-8") as f:
    if myxnli_seamless_m4t_back_translated_start_index_1 == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(myxnli_back_translated.iloc[myxnli_seamless_m4t_back_translated_start_index_1:].iterrows(), 
                       total=len(myxnli_back_translated) - myxnli_seamless_m4t_back_translated_start_index_1):
        original = row["sentence1_en"]
        translated, back_translated = back_translate_seamless_m4t(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'")
        translated = translated.replace('"', "'")
        back_translated = back_translated.replace('"', "'")
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")

In [ ]:
# set and display back-translated myXNLI dataset (sentences 1)
tmp_myxnli_seamless_m4t_back_translated_1 = load_tmp_df("myxnli_seamless_m4t_back_translated_1")
myxnli_back_translated["seamless_translated_s1"] = tmp_myxnli_seamless_m4t_back_translated_1["translated"]
myxnli_back_translated["seamless_back_translated_s1"] = tmp_myxnli_seamless_m4t_back_translated_1["back_translated"]
display(myxnli_back_translated.head())

In [ ]:
# Apply back-translation to the sentences 2 in myXNLI
print("Applying back-translation to myXNLI dataset (sentences 2)...")
myxnli_seamless_m4t_back_translated_start_index_2 = 0
with open("tmp/myxnli_seamless_m4t_back_translated_2.csv", mode="a", encoding="utf-8") as f:
    if myxnli_seamless_m4t_back_translated_start_index_2 == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(myxnli_back_translated.iloc[myxnli_seamless_m4t_back_translated_start_index_2:].iterrows(), 
                       total=len(myxnli_back_translated) - myxnli_seamless_m4t_back_translated_start_index_2):
        original = row["sentence2_en"]
        translated, back_translated = back_translate_seamless_m4t(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'")
        translated = translated.replace('"', "'")
        back_translated = back_translated.replace('"', "'")
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")

In [ ]:
# set and display back-translated myXNLI dataset (sentences 2)
tmp_myxnli_seamless_m4t_back_translated_2 = load_tmp_df("myxnli_seamless_m4t_back_translated_2")
myxnli_back_translated["seamless_translated_s2"] = tmp_myxnli_seamless_m4t_back_translated_2["translated"]
myxnli_back_translated["seamless_back_translated_s2"] = tmp_myxnli_seamless_m4t_back_translated_2["back_translated"]
display(myxnli_back_translated.head())

##### ALT dataset

In [ ]:
# Apply back-translation to the alt combined dataset
print("Applying back-translation to alt combined dataset...")
alt_combined_seamless_m4t_back_translated_start_index = 0
with open("tmp/alt_combined_seamless_m4t_back_translated.csv", mode="a", encoding="utf-8") as f:
    if alt_combined_seamless_m4t_back_translated_start_index == 0:
        # Write CSV header
        f.write("original,translated,back_translated\n")
    
    # Process rows and write results
    for index, row in tqdm(alt_combined_back_translated.iloc[alt_combined_seamless_m4t_back_translated_start_index:].iterrows(), 
                       total=len(alt_combined_back_translated) - alt_combined_seamless_m4t_back_translated_start_index):
        original = row["English_Sentence"]
        translated, back_translated = back_translate_seamless_m4t(original)
        
        # Replace double quotes with single quotes
        original = original.replace('"', "'")
        translated = translated.replace('"', "'")
        back_translated = back_translated.replace('"', "'")
        
        # Wrap text in double quotes if it contains a comma
        original = f'"{original}"' if ',' in original else original
        translated = f'"{translated}"' if ',' in translated else translated
        back_translated = f'"{back_translated}"' if ',' in back_translated else back_translated
        
        # Write to CSV
        if translated and back_translated:
            f.write(f"{original},{translated},{back_translated}\n")
        else:
            f.write(f"{original},,\n")

Applying back-translation to alt combined dataset...


  0%|          | 0/19166 [00:00<?, ?it/s]

In [ ]:
# set and display back-translated combined ALT dataset
tmp_alt_combined_seamless_m4t_back_translated = load_tmp_df("alt_combined_seamless_m4t_back_translated")
alt_combined_back_translated["seamless_translated"] = tmp_alt_combined_seamless_m4t_back_translated["translated"]
alt_combined_back_translated["seamless_back_translated"] = tmp_alt_combined_seamless_m4t_back_translated["back_translated"]
display(alt_combined_back_translated.head())

#### Save Back-Translated

In [ ]:
# save back-translated myXNLI dataset and combined ALT dataset
save_gen_df(myxnli_back_translated, "myxnli_back_translated")
save_gen_df(alt_combined_back_translated, "alt_combined_back_translated")

### 5. Pseudo-Parallel Corpus Creation

This step involves aligning monolingual English and Burmese text from the datasets to create pseudo-parallel corpora by using models (`sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2`). 
Semantic similarity methods are used to identify pairs of sentences with similar meanings. 
The resulting aligned corpus enhances the dataset and is valuable for low-resource language modeling.

In [8]:
# load normalized myXNLI and Combined ALT  dataset
myxnli_corpus = load_gen_df("myxnli_normalized")
alt_combined_corpus = load_gen_df("alt_combined_normalized")

In [9]:
# Extract monolingual text
corpus_english_sentences = list(myxnli_corpus["sentence1_en"]) + list(myxnli_corpus["sentence2_en"]) + list(alt_combined_corpus["English_Sentence"])
corpus_burmese_sentences = list(myxnli_corpus["sentence1_my"]) + list(myxnli_corpus["sentence2_my"]) + list(alt_combined_corpus["Burmese_Sentence"])

#### sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

In [10]:
# Load pretrained multilingual embedding model
minilm_embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
minilm_embedding_model = SentenceTransformer(minilm_embedding_model_name, device=device)

##### Generate Embeddings

In [11]:
# File paths for embeddings and checkpoints
minilm_corpus_english_embedding_file = "gen/minilm_corpus_english_embeddings.npy"
minilm_corpus_burmese_embedding_file = "gen/minilm_corpus_burmese_embeddings.npy"
minilm_corpus_embedding_checkpoint_file = "tmp/minilm_corpus_embedding_checkpoint.txt"

In [12]:
# Load existing progress if available
if os.path.exists("gen/minilm_pseudo_parallel_corpus.csv"):
    progress_df = load_gen_df("minilm_pseudo_parallel_corpus")
    minilm_corpus_aligned_pairs = progress_df.to_dict(orient="records")
else:
    minilm_corpus_aligned_pairs = []

In [13]:
# Initialize progress
if os.path.exists(minilm_corpus_english_embedding_file) and os.path.exists(minilm_corpus_burmese_embedding_file) and os.path.exists(minilm_corpus_embedding_checkpoint_file):
    print("Loading existing embeddings and checkpoint...")
    minilm_corpus_english_embeddings = np.load(minilm_corpus_english_embedding_file)
    minilm_corpus_burmese_embeddings = np.load(minilm_corpus_burmese_embedding_file)
    with open(minilm_corpus_embedding_checkpoint_file, "r") as f:
        minilm_corpus_embeddings_start_index = int(f.read().strip())
else:
    print("Starting fresh embedding computation...")
    minilm_corpus_english_embeddings = None
    minilm_corpus_burmese_embeddings = None
    minilm_corpus_embeddings_start_index = 0

Loading existing embeddings and checkpoint...


In [14]:
# Process sentences in batches and save incrementally
minilm_corpus_embeddings_batch_size = 100
for i in tqdm(range(minilm_corpus_embeddings_start_index, len(corpus_english_sentences), minilm_corpus_embeddings_batch_size)):
    
    # Get current batch of sentences
    english_batch = corpus_english_sentences[i:i+minilm_corpus_embeddings_batch_size]
    burmese_batch = corpus_burmese_sentences[i:i+minilm_corpus_embeddings_batch_size]
    
    # Compute embeddings
    english_batch_embeddings = minilm_embedding_model.encode(english_batch)
    burmese_batch_embeddings = minilm_embedding_model.encode(burmese_batch)
    
    # Merge with existing embeddings
    if minilm_corpus_english_embeddings is None:
        minilm_corpus_english_embeddings = english_batch_embeddings
        minilm_corpus_burmese_embeddings = burmese_batch_embeddings
    else:
        minilm_corpus_english_embeddings = np.vstack([minilm_corpus_english_embeddings, english_batch_embeddings])
        minilm_corpus_burmese_embeddings = np.vstack([minilm_corpus_burmese_embeddings, burmese_batch_embeddings])

    # Save progress
    np.save(minilm_corpus_english_embedding_file, minilm_corpus_english_embeddings)
    np.save(minilm_corpus_burmese_embedding_file, minilm_corpus_burmese_embeddings)
    with open(minilm_corpus_embedding_checkpoint_file, "w") as f:
        f.write(str(i + minilm_corpus_embeddings_batch_size))

print("Embedding computation completed.")

0it [00:00, ?it/s]

Embedding computation completed.


##### Semantic Similarity and Filter High-Similarity

In [15]:
# chunk and max
minilm_corpus_chunk = 5
minilm_corpus_max = 10000 * minilm_corpus_chunk

In [16]:
# files
minilm_corpus_checkpoint_file = f"tmp/minilm_pseudo_parallel_last_index_{minilm_corpus_chunk}.txt"

In [17]:
# Load existing progress if available
if os.path.exists(f"tmp/minilm_pseudo_parallel_corpus_{minilm_corpus_chunk}.csv"):
    corpus_progress_df = load_tmp_df(f"minilm_pseudo_parallel_corpus_{minilm_corpus_chunk}")
    minilm_corpus_aligned_pairs = corpus_progress_df.to_dict(orient="records")
else:
    minilm_corpus_aligned_pairs = []

In [18]:
# Determine the starting index
if os.path.exists(minilm_corpus_checkpoint_file):
    with open(minilm_corpus_checkpoint_file, "r") as f:
        minilm_corpus_start_index = int(f.read().strip())
else:
    minilm_corpus_start_index = minilm_corpus_max - 10000

print(f"Start Index: {minilm_corpus_start_index}")

Start Index: 40000


In [20]:
# Align sentences based on similarity threshold
minilm_threshold = 0.8
minilm_corpus_batch_size = 1000

# Process sentences in batches
for batch_start in tqdm(range(minilm_corpus_start_index, minilm_corpus_max, minilm_corpus_batch_size)):
    batch_end = min(batch_start + minilm_corpus_batch_size, len(corpus_english_sentences))  # Ensure no overflow
    english_batch = corpus_english_sentences[batch_start:batch_end]

    for i, english_sentence in tqdm(enumerate(english_batch), desc=f"Processing batch {batch_start}", total=len(english_batch), leave=False):
        current_index = batch_start + i  # Absolute index in the full corpus
        for j, burmese_sentence in tqdm(enumerate(corpus_burmese_sentences), desc=f"Aligning sentences for index {current_index}", total=len(corpus_burmese_sentences), leave=False, mininterval=1.0):
            # Compute similarity score
            similarity_score = util.cos_sim(minilm_corpus_english_embeddings[current_index], minilm_corpus_burmese_embeddings[j]).item()
            if similarity_score > minilm_threshold:
                minilm_corpus_aligned_pairs.append({
                    "english": english_sentence,
                    "burmese": burmese_sentence,
                    "similarity_score": similarity_score
                })

    # Save progress incrementally after each batch
    progress_df = pd.DataFrame(minilm_corpus_aligned_pairs)
    save_tmp_df(progress_df, f"minilm_pseudo_parallel_corpus_{minilm_corpus_chunk}")

    # Update checkpoint file after each batch
    with open(minilm_corpus_checkpoint_file, "w") as f:
        f.write(str(batch_end))

print("Pseudo-parallel corpus creation completed.")

  0%|          | 0/10 [00:00<?, ?it/s]

Processing batch 40000:   0%|          | 0/1000 [00:00<?, ?it/s]

Aligning sentences for index 40000:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40001:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40002:   0%|          | 0/804530 [00:00<?, ?it/s]

...

Aligning sentences for index 40015:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40016:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40017:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40018:   0%|          | 0/804530 [00:00<?, ?it/s]

Aligning sentences for index 40019:   0%|          | 0/804530 [00:00<?, ?it/s]